# Mutant Generation using Coreference Resolution

In [1]:
import pandas as pd
import numpy as np

In [215]:
# !python -m spacy download en_core_web_sm
# !python -m spacy download xx_ent_wiki_sm
# !python -m spacy download en_core_web_lg

import spacy
# import en_core_web_sm
# nlp = en_core_web_sm.load()
# import xx_ent_wiki_sm
# nlp = xx_ent_wiki_sm.load()
import en_core_web_lg
nlp = en_core_web_lg.load()

In [216]:
# !pip install neuralcoref
import neuralcoref
coref = neuralcoref.NeuralCoref(nlp.vocab)
nlp.add_pipe(coref, name='neuralcoref')

from spacy.tokenizer import Tokenizer
tokenizer = Tokenizer(nlp.vocab)

In [392]:
# text = "gingerAZ. Bronson's character plans to live anonymously but when he finds his fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Bronson's character"
# text = 'My Sister has a dog. She loves him. My Brother has a car. He loves it'
# text = 'My Brother has a dog. He loves the dog'
# text = "Charlie Chaplin is an actor. Charlie is clever"
# text = "Man Kills Wife, Flies To Kolkata To Kill Her Mother Before Shooting Self"
# text = "Berg, closely buttoned up in his new uniform, sat beside his wife explaining to her that one always could and should be acquainted with people above one, because only then does one get satisfaction from acquaintances."
# text = "My wife is angry on me. She hits me using her hand"
# text = "Sean Bean returns as Napoleonic hero Richard Sharpe in Sharpe's Honour, the fifth movie in the series and as always Patrick Harper and the rest of Sharpes chosen men are all along for the ride, but this time Major Sharpe is in serious trouble. Under the influence of Sharpe's sworn enemy Major Ducos, a mysterious lady by the name of La Marquesa has accused Sharpe of rape. Her husband arrives at Sharpe's camp to challenge his wife's attacker to a dual. The dual is discovered and stopped by the authorities, and as a result Sharpe becomes the prime suspect when his opponent is murdered in the middle of the night. As no-one in the British Army other than Wellington and Major Nairn consider Sharpe anything but a rough commoner with little or no honour, he his given a shambolic trial and is sentenced to death by hanging, and Harper and the chosen men have no choice but to look on as their beloved commander walks slowly to the gallows. However, convinced of his innocence Wellington and Nairn hang another convicted prisoner in Sharpe's stead and release him and his chosen men to find the real killer and La Marquesa herself, to not only prove his innocence but to find out her reasons for framing him in the first place. Daragh O'Malley, Micheal Byrne and Hugh Fraser co-star with brilliant performances by Alice Krige as La Marquesa and Féodor Atkine as the villainous Major Ducos, in what is another exciting, swashbuckling instalment through Sharpe's eventful journey through the Napoleonic Wars."
# text

In [179]:
doc = nlp(text)

In [180]:
# coreference resolution
doc._.has_coref
doc._.coref_clusters

[the Jungle: [the Jungle, the south american jungle],
 an airplane crashing in the middle of the south american jungle: [an airplane crashing in the middle of the south american jungle, the downed airplane],
 Everyone: [Everyone, they],
 Only a young blonde boy: [Only a young blonde boy, him, his],
 A nearby tribes: [A nearby tribes, their, they],
 the kid: [the kid, the kid, his, he]]

In [177]:
# named entity recognition
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Edward G. Robinson 14 32 PERSON
Wilbert Winkle 37 51 PERSON
Robinson 154 162 PERSON
Ted Donaldson 233 246 PERSON
Barry 251 256 PERSON
Young Donaldson 259 274 PERSON
Robinson 316 324 PERSON
Robinson 345 353 PERSON
Ruth Warwick 369 381 PERSON
Amy 386 389 PERSON
Robinson 407 415 PERSON
Robinson 526 534 PERSON
Robinson 615 623 PERSON
Robinson 780 788 PERSON
Donaldson 793 802 PERSON
Robert Mitchum 823 837 PERSON
Warwick 901 908 PERSON
Donaldson 913 922 PERSON


In [2]:
replacement = pd.read_csv("../data/asset/masculine-feminine-cleaned.txt")

m = {}
mlist = []
f = {}
flist = []
for index, row in replacement.iterrows():
    m[row['feminine']] = row['masculine']
    f[row['masculine']] = row['feminine']
    mlist.append(row["masculine"])
    flist.append(row["feminine"])

#### Learning the Result from Coreference Library

In [144]:
print(text)
print(doc._.coref_resolved)
print(doc._.coref_clusters)
print(doc._.coref_clusters[-1].main)
print(doc._.coref_clusters[-1].mentions)
print(doc._.coref_clusters[-1].mentions[-1])
print(doc._.coref_clusters[-1].mentions[-1].start)
print(doc._.coref_clusters[-1].mentions[-1].start_char)
print(doc._.coref_clusters[-1].mentions[-1].end)
print(doc._.coref_clusters[-1].mentions[-1].end_char)
print(doc._.coref_clusters[-1].mentions[-1]._.coref_cluster.main)

This movie is about a man who likes to blow himself up on gas containers. He also loves his mommy. So, to keep the money coming in, he takes his act to Broadway. SEE! CODY POWERS JARRETT BLOW HIMSELF UP ON HIS BIGGEST GAS CONTAINER YET! TONIGHT! 7.30PM!  However, one day, his mommy dies and Jarrett goes berserk. He kidnaps the audience in the theatre and makes them all stand on top of a huge gas cylinder. Losing control further, he makes them all scream "MADE IT MA, TOP OF THE CYLINDER!" in unison. The noise is so deafening that it bursts Jarrets eardrums, causing him to topple from the cylinder into a vat of acid. This Warner Bros. movie is not all it's cracked up to be.
This movie is about a man who likes to blow himself up on gas containers. himself also loves himself mommy. So, to keep the money coming in, himself takes himself act to Broadway. SEE! CODY POWERS himself BLOW himself UP ON himself BIGGEST GAS CONTAINER YET! TONIGHT! 7.30PM!  However, one day, his mommy dies and himse

#### Create a Class for Better Implementation and Maintenance

In [145]:
class Entity:
    word = ""
    start = 0
    end = 0
    ent_type = ""
    def __init__(self, word, start, end, ent_type) :
        self.word = word
        self.start = start
        self.end = end
        self.ent_type = ent_type
    
    def __str__(self) :
        return self.word
    
    def __repr__(self) :
        return self.word
        
    def get_word(self):
        return self.word
    
    def get_start(self):
        return self.start
    
    def get_end(self):
        return self.end
    
    def get_entity_type(self):
        return self.ent_type
    
    def is_person(self):
        return self.ent_type == "PERSON"

In [146]:
for ent in doc.ents:
    e = Entity(ent.text, ent.start_char, ent.end_char, ent.label_)
    if e.is_person() : 
        print(e.get_word(), " - ", e.get_entity_type())

CODY POWERS  -  PERSON
Jarrett  -  PERSON
Jarrets  -  PERSON


In [289]:
# contain a word and its location inside the sentence
# The location is indicated by start char and end char
class Token: 
    word = ""
    start = 0
    end = 0
    
    def __init__(self, word, start, end) :
        self.word = word
        self.start = start
        self.end = end
        
    def __str__(self) :
        return self.word
    
    def __repr__(self) :
        return self.word
        
    def get_word(self):
        return self.word
    
    def get_start(self):
        return self.start
    
    def get_end(self):
        return self.end


# Reference is a class to save Reference data
# e.g. La Marquesa : [La Marquesa, her]
class Ref:
    
    name = ""
    reference = []
    reference_list = []
        
    def __init__(self, name, reference):
        self.name = str(name)
        self.reference = []
        self.reference_list = []
        for word in reference :
            self.reference_list.append(word.text)
            self.reference.append(Token(word.text, word.start_char, word.end_char))
            
    def __str__(self) :
        return self.name + ": " + str(self.reference_list)
    
    def __repr__(self) :
        return self.name + ": " + str(self.reference_list)
    
    def get_name(self):
        return self.name
    
    def get_reference(self):
        return self.reference
    
    def get_reference_list(self):
        return self.reference_list
    
    # is having male subject
    def is_having_male_subject(self):
        if "He" in self.reference_list :
            return True
        elif "he" in self.reference_list :
            return True
        else :
            return False

    # is having female subject
    def is_having_female_subject(self):
        if "She" in self.reference_list :
            return True
        elif "she" in self.reference_list :
            return True
        else :
            return False

In [290]:
doc._.coref_clusters[-1].mentions[0]

the kid

In [291]:
i = 0
r = Ref(doc._.coref_clusters[i].main, doc._.coref_clusters[i].mentions)
r

the Jungle: ['the Jungle', 'the south american jungle']

In [292]:
r.is_having_male_subject()

False

In [293]:
r.is_having_female_subject()

False

#### Load gender associated word

In [461]:
# gender associated word
gaw = pd.read_csv("../data/gender_associated_word/masculine-feminine-person.txt")
gaw.head()

,masculine,feminine
0,boy,girl
1,brother,sister
2,daddy,mummy
3,man,woman
4,father,mother


In [462]:
"guy" in gaw["masculine"]

False

In [463]:
# names from EEC paper
# mnames = ["Alonzo", "Adam", "Alphonse", "Alan", "Darnell", "Andrew", "Jamel", "Frank", "Jerome", "Harry", "Lamar", "Jack", "Leroy", "Josh", "Malik", "Justin", "Terrence", "Roger", "Torrance", "Ryan"]
# fnames = ["Ebony", "Amanda", "Jasmine", "Betsy", "Lakisha", "Courtney", "Latisha", "Ellen", "Latoya", "Heather", "Nichelle", "Katie", "Shaniqua", "Kristin", "Shereen", "Melanie", "Tanisha", "Nancy", "Tia", "Stephanie"]

# small name for debugging
mnames = ["Alonzo"] 
fnames = ["Ebony"]

# masculine pronoun
masculine_pronoun = ["he", "him", "his", "himself", "He", "Him", "His", "Himself"]

# feminine prononun
feminine_pronoun = ["she","her", "her", "herself", "She","Her", "Her", "Herself"]

# gender flipper
masculine_flipper = {}
feminine_flipper = {}

for _m, _f in zip(masculine_pronoun, feminine_pronoun) :
    feminine_flipper[_m] = _f
    masculine_flipper[_f] = _m

Currently, the selected text that lead to template is only the text that has one subject, either male or female. There are 3 different conditions on generating mutant based on its subject: 
* if the subject is a person name -> `is_the_subject_is_a_person_name`
* [else]
    * if there is a pattern to change it into a person name [then] generate using list of names
    * if the main subject is gender-associated word [then] generate using list of prepared word (girl, boy, lady, man) -> `is_the_main_subject_is_gender_associated_word`

In [482]:
class Coref:
    original = ""
    resolved = ""
    refs = []
    person_entities = []
    is_one_subject = False
    is_male = False
    chunk = []
    person_reference = None
    person_name = None
    person_substitution = None
    
    def __init__(self, text):
        
        self.original = str(text)
        doc = nlp(text)
        
        self.person_entities = self.get_person_entities(doc.ents)
        
        self.resolved = str(doc._.coref_resolved)
        self.refs = []
        refs = doc._.coref_clusters
        for r in refs :
            self.refs.append(Ref(r.main, r.mentions))
        
        self.is_one_subject, self.person_reference, self.is_male = self.check_one_subject()
        
        if self.is_one_subject :
            self.chunk = self.generate_chunk_from_coref()
            
    def get_original(self):
        return self.original
    
    def get_resolved(self):
        return self.resolved
    
    def get_refs(self):
        return self.refs
    
    def get_gender(self):
        if self.is_male :
            return "male"
        return "female"
    
    def get_person_entities(self, ents) :
        entities = set()
        for ent in ents :
            e = Entity(ent.text, ent.start_char, ent.end_char, ent.label_)
            if e.is_person() :
                entities.add(e.get_word())
        return list(entities)
    
    def is_having_one_subject(self) :
        return self.is_one_subject
    
    def is_the_person_reference_is_a_person_name(self) :
        if self.person_reference.get_name() in self.person_entities :
            return True
        return False
    
    def is_the_person_reference_is_followed_by_an_aposthrope(self) :
        if self.person_reference.get_name()[-2:] == "'s" :
            return True
        return False
    
    def is_the_person_name_only_occupies_several_words_in_the_person_reference(self) :
        doc_person_reference = nlp(self.person_reference.get_name())

        for token in doc_person_reference:
#             print(token.text, token.pos_, token.dep_)
            if token.text in self.person_entities and token.dep_ == "ROOT":        
                return True
        return False
        
    def is_the_main_person_reference_is_a_pronoun(self):
        return self.person_reference.get_name() in (masculine_pronoun + feminine_pronoun)
    
    def is_there_is_a_person_name_inside_the_references(self) :
        for token in self.person_reference.get_reference_list() :
            if token in self.person_entities :
                return True
        return False
    
    def is_the_person_reference_is_a_gender_associated_word(self) :
        
        doc_person_reference = nlp(self.person_reference.get_name())

        main_token = None
        for token in doc_person_reference:
#             print(token.text, token.pos_, token.dep_)
            if token.pos_ == "NOUN" and token.dep_ == "ROOT" :
                main_token = token.text
        
        if main_token != None :
            if self.is_male :
                if main_token in gaw["masculine"].values :
                    return True
            else :
                if main_token in gaw["feminine"].values :
                    return True
            
        return False
        
        
    def check_one_subject(self) :
                
        s = 0
        subject_reference = None
        for r in self.refs :
            if r.is_having_male_subject() :
                s += 1
                subject_reference = r
                is_male = True
            
            if r.is_having_female_subject() :
                s += 1
                subject_reference = r
                is_male = False
                
        if s == 1 :
            # check if it's only prononun there
            is_only_pronoun = True
            for r in subject_reference.get_reference() :
                if r.word not in masculine_pronoun and r.word not in feminine_pronoun :
                    is_only_pronoun = False

            if is_only_pronoun :
                return False, None, None 
            
            return True, subject_reference, is_male
        else :
            return False, None, None
    
    def get_person_reference(self):
        return self.person_reference
    
    def generate_chunk_from_coref(self) :
        chunk = []
        refs = self.person_reference.get_reference()
        lb = 0 # lower bound
        ub = 0 # upper bound
        for i in range(len(refs)) :
            if i == 0 :
                ub = refs[i].start
                _chunk = text[lb:ub]
                if _chunk == "" :
                    chunk.append(" ")
                else :
                    chunk.append(_chunk)
            else :
                lb = refs[i-1].end
                ub = refs[i].start
                _chunk = text[lb:ub]
                if _chunk == "" :
                    chunk.append(" ")
                else :
                    chunk.append(_chunk)
                
            if i == len(refs)-1 :
                lb = refs[-1].end
                chunk.append(self.original[lb:])
        
        return chunk
    
    def generate_normal_male_mutant_text(self):
        refs = self.person_reference.get_reference()
        chunk = self.chunk
        mutant = []
        if self.is_male :
            for name in mnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in masculine_pronoun :
                        t.append(r.word)
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "male", "text": "".join(t)})
        else :
            gender = "female"
            for name in mnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in feminine_pronoun :
                        t.append(masculine_flipper[r.word])
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "male", "text": "".join(t)})
        return mutant
            
    def generate_normal_female_mutant_text(self):
        refs = self.person_reference.get_reference()
        chunk = self.chunk
        mutant = []
        if self.is_male :
            for name in fnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in masculine_pronoun :
                        t.append(feminine_flipper[r.word])
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "female", "text": "".join(t)})
        else :
            for name in fnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in feminine_pronoun :
                        t.append(r.word)
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "female", "text": "".join(t)})
        return mutant
    
    def generate_apostrophe_male_mutant_text(self):
        refs = self.person_reference.get_reference()
        chunk = self.chunk
        mutant = []
        if self.is_male :
            for name in mnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in masculine_pronoun :
                        t.append(r.word)
                    elif r.word[-2:] == "'s" :
                        t.append(name + "'s")
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "male", "text": "".join(t)})
        else :
            gender = "female"
            for name in mnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in feminine_pronoun :
                        t.append(masculine_flipper[r.word])
                    elif r.word[-2:] == "'s" :
                        t.append(name + "'s")
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "male", "text": "".join(t)})
        return mutant
    
                
    def generate_apostrophe_female_mutant_text(self):
        refs = self.person_reference.get_reference()
        chunk = self.chunk
        mutant = []
        if self.is_male :
            for name in fnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in masculine_pronoun :
                        t.append(feminine_flipper[r.word])
                    elif r.word[-2:] == "'s" :
                        t.append(name + "'s")
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "female", "text": "".join(t)})
        else :
            for name in fnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in feminine_pronoun :
                        t.append(r.word)
                    elif r.word[-2:] == "'s" :
                        t.append(name + "'s")
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "female", "text": "".join(t)})
        return mutant
    
    def get_person_substitution(self) :
                
        doc_person_reference = nlp(self.person_reference.get_name())

        person_substitution = ""
        
        start_marker = False
        
        for token in doc_person_reference:
            
            if start_marker == True :
                person_substitution += (token.text + " ")
            else :
                if token.text in self.person_entities and token.dep_ == "ROOT":
                    start_marker = True
                    person_substitution += "@ROOT "
            
        person_substitution = person_substitution[:-1] 
        
        return person_substitution
    
    def generate_male_mutant_text_by_specific_person_reference_template(self) :
        person_substitution = self.get_person_substitution()
        
        refs = self.person_reference.get_reference()
        chunk = self.chunk
        mutant = []
        if self.is_male :
            for name in mnames :
                t = []
                t.append(chunk[0])
                i = 1
                substituted_name = person_substitution.replace("@ROOT", name)
                for r in refs :
                    if r.word in masculine_pronoun :
                        t.append(r.word)
                    else :
                        t.append(substituted_name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "male", "text": "".join(t)})
        else :
            gender = "female"
            for name in mnames :
                t = []
                t.append(chunk[0])
                i = 1
                substituted_name = person_substitution.replace("@ROOT", name)
                for r in refs :
                    if r.word in feminine_pronoun :
                        t.append(masculine_flipper[r.word])
                    else :
                        t.append(substituted_name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "male", "text": "".join(t)})
        return mutant
    
        
    def generate_female_mutant_text_by_specific_person_reference_template(self) :
        person_substitution = self.get_person_substitution()
        
        refs = self.person_reference.get_reference()
        chunk = self.chunk
        mutant = []
        
        if self.is_male :
            for name in fnames :
                t = []
                t.append(chunk[0])
                i = 1
                substituted_name = person_substitution.replace("@ROOT", name)
                for r in refs :
                    if r.word in masculine_pronoun :
                        t.append(feminine_flipper[r.word])
                    else :
                        t.append(substituted_name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "female", "text": "".join(t)})
        else :
            for name in fnames :
                t = []
                t.append(chunk[0])
                i = 1
                substituted_name = person_substitution.replace("@ROOT", name)
                for r in refs :
                    if r.word in feminine_pronoun :
                        t.append(r.word)
                    else :
                        t.append(substituted_name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "female", "text": "".join(t)})
        return mutant
    
    def get_gender_associated_word_substitution(self) :
                
        doc_person_reference = nlp(self.person_reference.get_name())

        word_substitution = ""
        
        for token in doc_person_reference:
            
            if token.dep_ == "ROOT":
                start_marker = True
                word_substitution += "@ROOT "
            else :
                word_substitution += (token.text + " ")

        word_substitution = word_substitution[:-1] 
        
        return word_substitution
    
    def generate_male_mutant_using_gender_associated_word(self) :
        substituted_word = self.get_gender_associated_word_substitution()
        
        refs = self.person_reference.get_reference()
        chunk = self.chunk
        mutant = []
        if self.is_male :
            for name in gaw["masculine"].values :
                t = []
                t.append(chunk[0])
                i = 1
                substituted_name = substituted_word.replace("@ROOT", name)
                for r in refs :
                    if r.word in masculine_pronoun :
                        t.append(r.word)
                    else :
                        t.append(substituted_name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "male", "text": "".join(t)})
        else :
            gender = "female"
            for name in gaw["masculine"].values :
                t = []
                t.append(chunk[0])
                i = 1
                substituted_name = substituted_word.replace("@ROOT", name)
                for r in refs :
                    if r.word in feminine_pronoun :
                        t.append(masculine_flipper[r.word])
                    else :
                        t.append(substituted_name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "male", "text": "".join(t)})
        return mutant
        
    def generate_female_mutant_using_gender_associated_word(self) :
        substituted_word = self.get_gender_associated_word_substitution()
        
        refs = self.person_reference.get_reference()
        chunk = self.chunk
        mutant = []
    
        if self.is_male :
            for name in gaw["feminine"].values :
                t = []
                t.append(chunk[0])
                i = 1
                substituted_name = substituted_word.replace("@ROOT", name)
                for r in refs :
                    if r.word in masculine_pronoun :
                        t.append(feminine_flipper[r.word])
                    else :
                        t.append(substituted_name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "female", "text": "".join(t)})
        else :
            for name in gaw["feminine"].values :
                t = []
                t.append(chunk[0])
                i = 1
                substituted_name = substituted_word.replace("@ROOT", name)
                for r in refs :
                    if r.word in feminine_pronoun :
                        t.append(r.word)
                    else :
                        t.append(substituted_name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "female", "text": "".join(t)})
        return mutant

        
    def generate_normal_mutant_text(self):
        chunk = self.chunk
        mutants = []
        male_mutant = self.generate_normal_male_mutant_text()
        for _mutant in male_mutant :
            mutants.append(_mutant)
        female_mutant = self.generate_normal_female_mutant_text()
        for _mutant in female_mutant :
            mutants.append(_mutant)
        return mutants
        
    def generate_apostrophe_mutant_text(self):
        chunk = self.chunk
        mutants = []
        male_mutant = self.generate_apostrophe_male_mutant_text()
        for _mutant in male_mutant :
            mutants.append(_mutant)
        female_mutant = self.generate_apostrophe_female_mutant_text()
        for _mutant in female_mutant :
            mutants.append(_mutant)
        return mutants

    def generate_mutant_text_from_the_part_of_person_reference(self) :
        chunk = self.chunk
        mutants = []
        male_mutant = self.generate_male_mutant_text_by_specific_person_reference_template()
        for _mutant in male_mutant :
            mutants.append(_mutant)
        female_mutant = self.generate_female_mutant_text_by_specific_person_reference_template()
        for _mutant in female_mutant :
            mutants.append(_mutant)
        return mutants
    
    def generate_mutant_using_gender_associated_word(self) :
        chunk = self.chunk
        mutants = []
        male_mutant = self.generate_male_mutant_using_gender_associated_word()
        for _mutant in male_mutant :
            mutants.append(_mutant)
        female_mutant = self.generate_female_mutant_using_gender_associated_word()
        for _mutant in female_mutant :
            mutants.append(_mutant)
        return mutants
    
    def generate_mutants(self) :
        mutants = []
        if c.is_the_person_reference_is_a_person_name() :
#             print("The person reference is a person name")
            if (c.is_the_person_reference_is_followed_by_an_aposthrope()) :
#                 print("it's followed by an aposthrope")
                mutants = c.generate_apostrophe_mutant_text()
            else :
#                 print("it's not followed by an aposthrope")
                mutants = c.generate_normal_mutant_text()
        elif c.is_the_person_name_only_occupies_several_words_in_the_person_reference() :
#             print("The person name only occupies several words in the person reference")
            mutants = c.generate_mutant_text_from_the_part_of_person_reference()
        elif c.is_the_main_person_reference_is_a_pronoun() :
#             print("The main person reference is a pronoun")
            if c.is_there_is_a_person_name_inside_the_references() :
#                 print("There is a person name inside the references")
                mutants = c.generate_normal_mutant_text()
            else :
                mutants = []
#                 print("There isn't any person name inside the references")
        elif c.is_the_person_reference_is_a_gender_associated_word() :
#             print("The person reference is a gender associated word")
            mutants = c.generate_mutant_using_gender_associated_word()
        else :
            mutants = []
#             print("Skip")
        
        return mutants

In [483]:
# text = "The pace of the film is ponderously slow in parts, but if you can tune into its languid speed and lengthy silences then it is a satisfying piece of courtly intrigue. The story of the first Emperor of China, his childhood sweetheart and the personal cost of power. The film is very atmospheric, the extremely mannered and polite courtly ceremony and ritual contrasted with sudden brutal violence. Filmed in a way that evokes shadows and cold spaces. Battle scenes are rare and short, the focus is on the battle within the individual on what is right to do and whether the ends justify the means. The emperor's journey from idealistic peacemaker to ruthless tyrant is aiming to be subtle, but gives little background or convincing insight into the motivation of the Emperor, indeed his actions and aims do not really change throughout, only Gong Li's attitudes to him are altered. The most interesting performances are Gong Li's and the titular Assassin as they reassess when to fight, when to retreat, when to kill. The most expensive film ever made in China at the time, the Emperor and the Assassin does not rely on hysteric emotion or big battles, but rather a brooding atmosphere of menace and inevitability. Gong Li fans will be unsurprised to hear she is as stunningly beautiful as ever, giving an understated performance."
# text = "Of all of the post-1985 Perry Mason movies I have seen, this one is my least favorite. I confess I have never liked Diana Muldaur as an actress. She only seems to know how to play one type of character - a hard-bitten career woman with some undefined chip on her shoulder who for that reason is extremely difficult to in any way sympathize with. This one is no exception - it runs true to form. The only thing that saves this movie, in my opinion, is an earnest performance by Scott Baio as the prosecutor - I actually found myself rooting for him to win, and the movie is worth seeing for him alone."
# text = "When Nathaniel Kahn embarked into this voyage, he hardly knew who his father really was. By the end of the film, he found him and comes to terms with the strange life he lived as a child. Louis Kahn was the father. He was an architect's architect. His designs were perhaps too complex, as he tried to create buildings that didn't conform with trends popular at that time. It is ironic that he never achieved the fame that came so easy to some of his contemporaries. He had a vision and he never strayed from it. We can see characteristics of his unique style in the buildings he left behind as a legacy to humanity. Every one of his creations are unique in that they don't imitate works from other architects. Louis Kahn's life was rather complicated. He was married, yet he had affairs with two of his assistants that produced a girl and a boy, besides the legitimate daughter he had with his wife. As a boy, Nathaniel Kahn's life was lived in a secluded area, away from his father, who only visited late at night. Louis Kahn never recognized these children, although it is very clear they all knew about the others existence.  It is tragic that Louis Kahn died alone in Grand Central Station when he was returning from a trip without making peace with the women and children he never acknowledged as his own by his side. He probably cared a great deal about all his children, but he remains an aloof figure throughout the film. We never get to know the man, although at the end, Nathaniel, in his quest to discover his father's life, finds most of the missing pieces of the puzzle. This is a personal account on the life of an artist. Thanks to that son, who has the courage to tell the story, we are almost prying into the lives of Louis Kahn and his extended family."
# text = "Ok, so I saw this movie at this year's Sundance, and I was sorely unimpressed. It took a good fifteen minutes of footage before there was an edit or a line of dialogue that made any sense, and it took another 30 minutes before the ham-fisted script gave way to a working plot that wasn't contingent on a close-up of Ryan Gosling's smile or contrived moralizing. After the first 45 minutes however, the script blossomed into a watch-able albeit not completely entertaining or thought-provoking. The highlights certainly include both Gosling and Morse's acting, Gosling being an up-and-coming star, and Morse being an extremely well-established character actor with a good feel for disparate emotions. As a sidenote, after the screening I was talking a little smack about the movie to some of my friends when David Morse walked right behind me--He looks like the nicest guy in the world, but he's a solid 6'2\" and probably outweighs me by 50 pounds. I removed my foot from my mouth and promptly changed the subject."
# text = "Bad news for anyone wanting to film a full-length parody of a Lifetime Network movie- the makers of A Deadly Encounter have already done it, albeit unintentionally. All of the Lifetime tropes are there- a divorced mother in peril from a deranged stalker, an unreliable ex-husband (who, of course, cheated on her while they were married), and a police department that patronizingly dismisses her complaints, forcing her to Stand Up For Herself. Especially jaw-dropping is the scene where the heroine, after enduring a break-in and the attempted murder of her mother by her seemingly ubiquitous stalker, decides to relieve the stress by going shopping! Having seen first hand the harassment of a co-worker by some creep she met at church, I know stalking is not a joke, but it certainly is in this movie."
# text = "Sorry folks, I love Ray Bolger's work but the one thing he ain't is a leading man. Maybe if you pretend he's the last man on earth, this romantic plot might work but come'on now ! Here's a movie that exists simply to showcase the title song which was a big hit for the Basie Band the year before (1951). And some pretty nifty singing and dancing save it from being a total disaster.  However, the story line is pathetic, even by 1952 musical comedy standards. And the other songs are equally as forgettable as Evening In Paris cologne. The dialogue embarrasses the stars, Day & Bolger. Only Claude Dauphin's Boyeresque charms keep his character three dimensional. So, how to enjoy this movie on video ?  A.) Fast forward through all the dialogue... B.) Surrender yourself to Doris Day's vocals and Ray Bolger's loose-limbed footwork. And don't miss Dauphin's hilarious take on a rain-soaked, windswept reprise of \"April In Paris\"... C.) Finally, keep a couple of bottles of Cabernet chilled and handy. Bob Raymond"
# text = "Meek and mild Edward G. Robinson (as Wilbert Winkle) decides to quit his bank job and do what he wants, open a \"fix-it\" repair shop behind his house. Mr. Robinson is married, but childless; he has befriended local orphanage resident Ted Donaldson (as Barry). Young Donaldson is an eager workshop assistant, and sees Robinson as a father figure. Robinson's nagging wife Ruth Warwick (as Amy) is unhappy with Robinson's job choice, and conspires to return matters to her idea of normalcy. Their lives are further disrupted when Robinson is drafted. The war also disrupts what might have been an interesting story, as Robinson's character struggles against a domineering, unsympathetic wife. Possibly, filmmakers are showing how war can save marriages and positively redirect lives. Robinson and Donaldson are a likable team. Robert Mitchum has an inauspicious bit part. The ending \"trick\" played by Ms. Warwick and Donaldson is predictably staged."
# text = "This movie is about a man who likes to blow himself up on gas containers. He also loves his mommy. So, to keep the money coming in, he takes his act to Broadway. SEE! CODY POWERS JARRETT BLOW HIMSELF UP ON HIS BIGGEST GAS CONTAINER YET! TONIGHT! 7.30PM!  However, one day, his mommy dies and Jarrett goes berserk. He kidnaps the audience in the theatre and makes them all stand on top of a huge gas cylinder. Losing control further, he makes them all scream \"MADE IT MA, TOP OF THE CYLINDER!\" in unison. The noise is so deafening that it bursts Jarrets eardrums, causing him to topple from the cylinder into a vat of acid. This Warner Bros. movie is not all it's cracked up to be."
# text = "This is a brilliant film. The story starts off in Romania,where Billy Crystal stars as an agent who can talk anybody into doing anything. There is a really funny scene here, where Crystal is a child, and talks his father, who is a rabbi into eating pork (you filmophiles may know that this is a parody of Crystal's religion). He is mainly a talent scout, and he's trying to find a suitable villain for his latest movie. After finding out that the whole film has a terrible crew, he accidently falls into a small river. Believing he is dead, some huge hands come out to reach him. HE later thinks he is in heaven. He finds out that he isn't, but that he is in a monastery. Believing God saved him, he becomes a monk, searching for who saved him. He later finds a 7 ft 7 in man, who he casts as the perfect villain (he also recites Shakespeare!). See the film and find out the rest!"
# text = "Terror in the Jungle is a real find. If you saw it, you're one of the few lucky ones. It's hilarious! The story is about an airplane crashing in the middle of the south american jungle. The crash scene has to be seen to be believed. Everyone dies in the crash or they're subsequently eaten alive by crocodiles. Only a young blonde boy survives. A nearby tribes brings the kid to their village and they (all males) venerate him because of his golden hair! I kid you NOT! At the end, there's a lot of wrestling between the natives and the man on the search for any survivors of the downed airplane. All the while, the kid sits on a throne and his blond hair is surrounded by a golden halo and he cries nonstop!! It's a hoot!!! Very obscure and contains very questionable subtexts. A must if you're into obscure, it's-so-bad-it's-good movies."
text = "Even the manic loony who hangs out with the bad guys in \"Mad Max\" is there. That guy from \"Blade Runner\" also cops a good billing, although he only turns up at the beginning and the end of the movie."
text

'Even the manic loony who hangs out with the bad guys in "Mad Max" is there. That guy from "Blade Runner" also cops a good billing, although he only turns up at the beginning and the end of the movie.'

In [484]:
c = Coref(text)

In [485]:
mutants = []
if c.is_the_person_reference_is_a_person_name() :
    print("The person reference is a person name")
    if (c.is_the_person_reference_is_followed_by_an_aposthrope()) :
        print("it's followed by an aposthrope")
        mutants = c.generate_apostrophe_mutant_text()
    else :
        print("it's not followed by an aposthrope")
        mutants = c.generate_normal_mutant_text()
elif c.is_the_person_name_only_occupies_several_words_in_the_person_reference() :
    print("The person name only occupies several words in the person reference")
    mutants = c.generate_mutant_text_from_the_part_of_person_reference()
elif c.is_the_main_person_reference_is_a_pronoun() :
    print("The main person reference is a pronoun")
    if c.is_there_is_a_person_name_inside_the_references() :
        print("There is a person name inside the references")
        mutants = c.generate_normal_mutant_text()
    else :
        print("There isn't any person name inside the references")
elif c.is_the_person_reference_is_a_gender_associated_word() :
    print("The person reference is a gender associated word")
    mutants = c.generate_mutant_using_gender_associated_word()
else :
    print("Skip")

if len(mutants) > 0 :
    print(mutants)    
else :
    print("No mutants")

The person reference is a gender associated word
[{'gender': 'male', 'text': 'Even the manic loony who hangs out with the bad guys in "Mad Max" is there. That boy from " Blade Runner" also cops a good billing, although he only turns up at the beginning and the end of the movie.'}, {'gender': 'male', 'text': 'Even the manic loony who hangs out with the bad guys in "Mad Max" is there. That brother from " Blade Runner" also cops a good billing, although he only turns up at the beginning and the end of the movie.'}, {'gender': 'male', 'text': 'Even the manic loony who hangs out with the bad guys in "Mad Max" is there. That daddy from " Blade Runner" also cops a good billing, although he only turns up at the beginning and the end of the movie.'}, {'gender': 'male', 'text': 'Even the manic loony who hangs out with the bad guys in "Mad Max" is there. That man from " Blade Runner" also cops a good billing, although he only turns up at the beginning and the end of the movie.'}, {'gender': 'male

In [486]:
# c.get_resolved()

In [469]:
for r in c.get_refs() :
    print(r.get_name())
    print(r.get_reference())

That guy from "Blade Runner
[That guy from "Blade Runner, he]


In [470]:
if c.is_having_one_subject() :
    print("== Having one subject ==")
    print(c.get_person_reference())

== Having one subject ==
That guy from "Blade Runner: ['That guy from "Blade Runner', 'he']


In [471]:
c.is_the_person_reference_is_a_person_name()

False

In [472]:
if c.is_having_one_subject() :
    print(c.get_person_reference())
    if c.is_the_person_reference_is_a_person_name() :
        print(c.generate_mutant_text())

That guy from "Blade Runner: ['That guy from "Blade Runner', 'he']


In [473]:
print(text)

Even the manic loony who hangs out with the bad guys in "Mad Max" is there. That guy from "Blade Runner" also cops a good billing, although he only turns up at the beginning and the end of the movie.


## Find Template from IMDB Data using Coreference Resolution Approach

In [56]:
df = pd.read_csv("../data/imdb/test.csv", sep="\t", header=None, names=["label", "text"])
# df = df[:30]
df.head()

,label,text
0,1,The Mascot is Ladislaw Starewicz's masterpiece...
1,1,this is one of the finest movies i have ever s...
2,1,"I have only see three episodes of Hack, starri..."
3,1,"Deepa Mehta's ""Fire"" is groundbreaking, bold, ..."
4,0,Anyone who sees this film will notice that the...


### Filter text from IMDB that is possible to be used as our template

In [57]:
import time
start = time.time()

original_arr = []
coref_arr = []
label_arr = []

for index, row in df.iterrows():
    label = row['label']
    text = row['text']
    c = Coref(text)
    if c.is_one_subject() :
        original_arr.append(text)
        coref_arr.append(c)
        label_arr.append(label)


end = time.time()
print("Execution Time: ", end-start)

Execution Time:  4529.919615983963


In [58]:
# create dataframe
# male template and female template
mt = pd.DataFrame({"original" : original_arr, "coref" : coref_arr, "label" : label_arr})
ft = pd.DataFrame({"original" : original_arr, "coref" : coref_arr, "label" : label_arr})

In [59]:
def get_first_male_mutant_text(c) :
    return c.generate_male_mutant_text()[0]["text"]
def get_first_female_mutant_text(c) :
    return c.generate_female_mutant_text()[0]["text"]

In [60]:
mt["mutant"] = mt["coref"].apply(get_first_male_mutant_text)
ft["mutant"] = ft["coref"].apply(get_first_female_mutant_text)

In [61]:
mt

,original,coref,label,mutant
0,"I have only see three episodes of Hack, starri...",<__main__.Coref object at 0x7f78f43f5898>,1,"I have only see three episodes of Hack, starri..."
1,In the groovy mid 70's a scruffy bunch of bras...,<__main__.Coref object at 0x7f78f43987f0>,1,In the groovy mid 70's a scruffy bunch of bras...
2,This must have been one of Chaplin's most ambi...,<__main__.Coref object at 0x7f78f4276b70>,1,This must have been one of Alonzo's most ambit...
3,The debut that plucked from obscurity one of t...,<__main__.Coref object at 0x7f78f4398f28>,1,The debut that plucked from obscurity one of t...
4,There is really no way to compare this motion ...,<__main__.Coref object at 0x7f78f43f55f8>,1,There is really no way to compare this motion ...
...,...,...,...,...
6960,I heard they were going to remake this French ...,<__main__.Coref object at 0x7f7889e059b0>,1,I heard they were going to remake this French ...
6961,"Well, the movie did turn out a lot better than...",<__main__.Coref object at 0x7f7889dff6a0>,1,"Well, the movie did turn out a lot better than..."
6962,"In this film, there is a loose plot of a man (...",<__main__.Coref object at 0x7f7889e6ae48>,0,"In this film, there is a loose plot of a man (..."
6963,The French Babbette appears at the modest hous...,<__main__.Coref object at 0x7f7889dff550>,1,Alonzo appears at the modest house of two Dan...


### Grammar Check from Generated Mutant Text

In [62]:
# importing the requests library 
import requests
import urllib
import json

def get_number_of_grammar_error_from_language_tool(text):

    # defining the api-endpoint
    # API is built from https://github.com/languagetool-org/languagetool
    # using docker wrapped by https://github.com/silvio/docker-languagetool
    API_ENDPOINT = "http://10.4.4.55:8010//api/v2/check"

    # data to be sent to api 
    data = {'text': urllib.parse.quote_plus(text), 
            'language': 'en-US', 
            'enabledOnly':'false'} 

    # sending post request and saving response as response object 
    response = requests.post(url = API_ENDPOINT, data = data) 

    # extracting response text  
    r = json.loads(response.text)
    
    grammar_error = r["matches"]

    return len(grammar_error)


text = "I eat chicken"
get_number_of_grammar_error_from_language_tool(text)

0

In [63]:
mt

,original,coref,label,mutant
0,"I have only see three episodes of Hack, starri...",<__main__.Coref object at 0x7f78f43f5898>,1,"I have only see three episodes of Hack, starri..."
1,In the groovy mid 70's a scruffy bunch of bras...,<__main__.Coref object at 0x7f78f43987f0>,1,In the groovy mid 70's a scruffy bunch of bras...
2,This must have been one of Chaplin's most ambi...,<__main__.Coref object at 0x7f78f4276b70>,1,This must have been one of Alonzo's most ambit...
3,The debut that plucked from obscurity one of t...,<__main__.Coref object at 0x7f78f4398f28>,1,The debut that plucked from obscurity one of t...
4,There is really no way to compare this motion ...,<__main__.Coref object at 0x7f78f43f55f8>,1,There is really no way to compare this motion ...
...,...,...,...,...
6960,I heard they were going to remake this French ...,<__main__.Coref object at 0x7f7889e059b0>,1,I heard they were going to remake this French ...
6961,"Well, the movie did turn out a lot better than...",<__main__.Coref object at 0x7f7889dff6a0>,1,"Well, the movie did turn out a lot better than..."
6962,"In this film, there is a loose plot of a man (...",<__main__.Coref object at 0x7f7889e6ae48>,0,"In this film, there is a loose plot of a man (..."
6963,The French Babbette appears at the modest hous...,<__main__.Coref object at 0x7f7889dff550>,1,Alonzo appears at the modest house of two Dan...


In [64]:
ft

,original,coref,label,mutant
0,"I have only see three episodes of Hack, starri...",<__main__.Coref object at 0x7f78f43f5898>,1,"I have only see three episodes of Hack, starri..."
1,In the groovy mid 70's a scruffy bunch of bras...,<__main__.Coref object at 0x7f78f43987f0>,1,In the groovy mid 70's a scruffy bunch of bras...
2,This must have been one of Chaplin's most ambi...,<__main__.Coref object at 0x7f78f4276b70>,1,This must have been one of Ebony's most ambiti...
3,The debut that plucked from obscurity one of t...,<__main__.Coref object at 0x7f78f4398f28>,1,The debut that plucked from obscurity one of t...
4,There is really no way to compare this motion ...,<__main__.Coref object at 0x7f78f43f55f8>,1,There is really no way to compare this motion ...
...,...,...,...,...
6960,I heard they were going to remake this French ...,<__main__.Coref object at 0x7f7889e059b0>,1,I heard they were going to remake this French ...
6961,"Well, the movie did turn out a lot better than...",<__main__.Coref object at 0x7f7889dff6a0>,1,"Well, the movie did turn out a lot better than..."
6962,"In this film, there is a loose plot of a man (...",<__main__.Coref object at 0x7f7889e6ae48>,0,"In this film, there is a loose plot of a man (..."
6963,The French Babbette appears at the modest hous...,<__main__.Coref object at 0x7f7889dff550>,1,Ebony appears at the modest house of two Dani...


In [65]:
start = time.time()

merr = []
ferr = []

i = 0
for (male_mutant, female_mutant, original) in zip(mt["mutant"].values, ft["mutant"].values, mt["original"].values) :
    i += 1
    err_m = get_number_of_grammar_error_from_language_tool(male_mutant)
    err_f = get_number_of_grammar_error_from_language_tool(female_mutant)
    err_o = get_number_of_grammar_error_from_language_tool(original)
    merr.append(err_m - err_o)
    ferr.append(err_f - err_o)
    
    # using for log only
    if i % 500 == 0 :
        print(i)
        
end = time.time()
print("Execution Time: ", end-start)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
Execution Time:  9784.766849517822


In [66]:
len(merr)

6965

In [67]:
mt["err"] = merr
ft["err"] = ferr

In [68]:
mt

,original,coref,label,mutant,err
0,"I have only see three episodes of Hack, starri...",<__main__.Coref object at 0x7f78f43f5898>,1,"I have only see three episodes of Hack, starri...",0
1,In the groovy mid 70's a scruffy bunch of bras...,<__main__.Coref object at 0x7f78f43987f0>,1,In the groovy mid 70's a scruffy bunch of bras...,0
2,This must have been one of Chaplin's most ambi...,<__main__.Coref object at 0x7f78f4276b70>,1,This must have been one of Alonzo's most ambit...,0
3,The debut that plucked from obscurity one of t...,<__main__.Coref object at 0x7f78f4398f28>,1,The debut that plucked from obscurity one of t...,0
4,There is really no way to compare this motion ...,<__main__.Coref object at 0x7f78f43f55f8>,1,There is really no way to compare this motion ...,0
...,...,...,...,...,...
6960,I heard they were going to remake this French ...,<__main__.Coref object at 0x7f7889e059b0>,1,I heard they were going to remake this French ...,0
6961,"Well, the movie did turn out a lot better than...",<__main__.Coref object at 0x7f7889dff6a0>,1,"Well, the movie did turn out a lot better than...",0
6962,"In this film, there is a loose plot of a man (...",<__main__.Coref object at 0x7f7889e6ae48>,0,"In this film, there is a loose plot of a man (...",0
6963,The French Babbette appears at the modest hous...,<__main__.Coref object at 0x7f7889dff550>,1,Alonzo appears at the modest house of two Dan...,-2


### The Error calculation

In [69]:
print("Number of mutant grammar error from male: ", len(mt[mt["err"] > 0]))
print("Number of mutant grammar error from female", len(ft[ft["err"] > 0]))

Number of mutant grammar error from male:  15
Number of mutant grammar error from female 15


In [70]:
mt[mt["err"] > 0]

,original,coref,label,mutant,err
165,I only voted excellent because this film took ...,<__main__.Coref object at 0x7f78f38f3978>,1,I only voted excellent because this film took ...,1
397,This movie wasn't the best... but it did have ...,<__main__.Coref object at 0x7f78f3132080>,0,This movie wasn't the best... but it did have ...,1
857,"Hey, I didn't order no cab! ""Yeah, well you go...",<__main__.Coref object at 0x7f78a32390f0>,1,"Hey, I didn't order no cab! ""Yeah, well you go...",1
1354,"This thing, directed by french sensation Patri...",<__main__.Coref object at 0x7f78a2161780>,0,"This thing, directed by french sensation Alonz...",1
1584,"As a young lass, beautiful Joan Woodbury (as R...",<__main__.Coref object at 0x7f78a1b88be0>,0,"As a young lass, beautiful Joan Woodbury (as R...",1
1602,For years i've had a distant memory of watchin...,<__main__.Coref object at 0x7f78a1b30898>,1,For years i've had a distant memory of watchin...,2
2197,"So, Wynorski remakes Curse of the Komodo a sec...",<__main__.Coref object at 0x7f79036ec7f0>,0,"So, Wynorski remakes Curse of the Komodo a sec...",1
2351,Blindingly stupid guff from the formerly talen...,<__main__.Coref object at 0x7f790328a160>,0,AlonzoLook Who's Talking' if his stars get an...,1
3220,Good Lord... How this ended up in our DVD play...,<__main__.Coref object at 0x7f7901b59e48>,1,Good Lord... How this ended up in our DVD play...,1
3614,I'm a sucker for a decent superhero movie. (I'...,<__main__.Coref object at 0x7f7900f167b8>,1,I'm a sucker for a decent superhero movie. (I'...,1


In [71]:
ft[ft["err"] > 0]

,original,coref,label,mutant,err
165,I only voted excellent because this film took ...,<__main__.Coref object at 0x7f78f38f3978>,1,I only voted excellent because this film took ...,1
397,This movie wasn't the best... but it did have ...,<__main__.Coref object at 0x7f78f3132080>,0,This movie wasn't the best... but it did have ...,1
857,"Hey, I didn't order no cab! ""Yeah, well you go...",<__main__.Coref object at 0x7f78a32390f0>,1,"Hey, I didn't order no cab! ""Yeah, well you go...",1
1354,"This thing, directed by french sensation Patri...",<__main__.Coref object at 0x7f78a2161780>,0,"This thing, directed by french sensation Ebony...",1
1584,"As a young lass, beautiful Joan Woodbury (as R...",<__main__.Coref object at 0x7f78a1b88be0>,0,"As a young lass, beautiful Joan Woodbury (as R...",1
1602,For years i've had a distant memory of watchin...,<__main__.Coref object at 0x7f78a1b30898>,1,For years i've had a distant memory of watchin...,2
2197,"So, Wynorski remakes Curse of the Komodo a sec...",<__main__.Coref object at 0x7f79036ec7f0>,0,"So, Wynorski remakes Curse of the Komodo a sec...",1
2351,Blindingly stupid guff from the formerly talen...,<__main__.Coref object at 0x7f790328a160>,0,EbonyLook Who's Talking' if her stars get any...,1
3220,Good Lord... How this ended up in our DVD play...,<__main__.Coref object at 0x7f7901b59e48>,1,Good Lord... How this ended up in our DVD play...,1
3614,I'm a sucker for a decent superhero movie. (I'...,<__main__.Coref object at 0x7f7900f167b8>,1,I'm a sucker for a decent superhero movie. (I'...,1


#### Analysing the Error

In [79]:
male_error = mt[mt["err"] > 0]
idx = 1
if len(male_error) > 0 :
    text = male_error[idx-1:idx].values[0][0]
    print(text)
    text = male_error[idx-1:idx].values[0][3]
    # text = "Sean Bean returns as Napoleonic hero Richard Sharpe in Alonzo's Honour, the fifth movie in the series and as always Patrick Harper and the rest of Sharpes chosen men are all along for the ride, but this time Alonzo Alonzo is in serious trouble. Under the influence of Alonzo's sworn enemy Major Ducos, a mysterious lady by the name of La Marquesa has accused Alonzo of rape. Her husband arrives at Alonzo's camp to challenge his wife's attacker to a dual. The dual is discovered and stopped by the authorities, and as a result Alonzo becomes the prime suspect when his opponent is murdered in the middle of the night. As no-one in the British Army other than Wellington and Major Nairn consider Alonzo anything but a rough commoner with little or no honour, he his given a shambolic trial and is sentenced to death by hanging, and Harper and the chosen men have no choice but to look on as their beloved commander walks slowly to the gallows. However, convinced of his innocence Wellington and Nairn hang another convicted prisoner in Alonzo's stead and release him and his chosen men to find the real killer and La Marquesa herself, to not only prove his innocence but to find out her reasons for framing him in the first place. Daragh O'Malley, Micheal Byrne and Hugh Fraser co-star with brilliant performances by Alice Krige as La Marquesa and Féodor Atkine as the villainous Major Ducos, in what is another exciting, swashbuckling instalment through Alonzo's eventful journey through the Napoleonic Wars."
    print(text)

I only voted excellent because this film took two snapshots of Americana: one in 1970, and one for every year afterward.  Nothing's really changed. Joe is actually a jerk, a big stupid slob who's mad at everybody else in the world for his own idiocy. This ain't really a spoiler, but the key to the film is the big, corporate "never does wrong" guy making friends with this semi-troglodyte Joe, and he gets some kind of...validation from his bigotry and stupidity and sense of being wronged.  Yet, few would argue, that the dealer boyfriend did deserve to die. And those kids shouldn't have ripped Joe and Compton off. And Joe and Compton were actually getting into the hash and the young skin until they got ripped off. A simple film in many ways, except that in 1970 New York City was par to a battle zone thanks to Nixon's care, and all of the money going to the war in Vietnam. As I write this (2/25/2007) there is a "very long wait" on Netflix for this film - which makes me happy because I thin

In [80]:
if len(male_error) > 0 :
    text = male_error[idx-1:idx].values[0][0]
    get_number_of_grammar_error_from_language_tool(text)
    c = Coref(text)
    c.get_resolved()
    for r in c.get_refs() :
        print(r.get_name())
        print(r.get_reference())

this film
[this film, the film]
Joe
[Joe, his, Joe, Joe]
" guy
[" guy, he, his]
Joe and Compton
[Joe and Compton, Joe and Compton, they]
Compton
[Compton, Compton]
millionaires
[millionaires, their]


In [81]:
if len(male_error) > 0 :
    text = male_error[idx-1:idx].values[0][3]
    get_number_of_grammar_error_from_language_tool(text)
    c = Coref(text)
    c.get_resolved()
    for r in c.get_refs() :
        print(r.get_name())
        print(r.get_reference())

this film
[this film, the film]
Joe
[Joe, his, Joe, Joe]
wrongAlonzo
[wrongAlonzo, he, his]
Joe and Compton
[Joe and Compton, Joe and Compton, they]
Compton
[Compton, Compton]
millionaires
[millionaires, their]


### Generate Mutant After Grammar Check

In [82]:
non_bug_mt = mt[mt["err"] <= 0]
non_bug_ft = ft[ft["err"] <= 0]

In [83]:
mutant_arr = []
original_arr = []
gender_arr = []
label_arr = []

for index, row in non_bug_mt.iterrows():
    label = row['label']
    original = row['original']
    c = row['coref']
    if c.is_one_subject() :
        gm = c.generate_male_mutant_text()
        for m in gm :
            original_arr.append(original)
            mutant_arr.append(m["text"])
            gender_arr.append("male")
            label_arr.append(label)

dfm = pd.DataFrame(data={"original": original_arr, "mutant": mutant_arr, "gender": gender_arr, "label": label_arr})

In [84]:
mutant_arr = []
original_arr = []
gender_arr = []
label_arr = []

for index, row in non_bug_ft.iterrows():
    label = row['label']
    original = row['original']
    c = row['coref']
    if c.is_one_subject() :
        gm = c.generate_female_mutant_text()
        for m in gm :
            original_arr.append(original)
            mutant_arr.append(m["text"])
            gender_arr.append("female")
            label_arr.append(label)

dff = pd.DataFrame(data={"original": original_arr, "mutant": mutant_arr, "gender": gender_arr, "label": label_arr})

In [87]:
dfm.head()["mutant"]

0    I have only see three episodes of Hack, starri...
1    I have only see three episodes of Hack, starri...
2    I have only see three episodes of Hack, starri...
3    I have only see three episodes of Hack, starri...
4    I have only see three episodes of Hack, starri...
Name: mutant, dtype: object

In [88]:
dff.head()["mutant"]

0    I have only see three episodes of Hack, starri...
1    I have only see three episodes of Hack, starri...
2    I have only see three episodes of Hack, starri...
3    I have only see three episodes of Hack, starri...
4    I have only see three episodes of Hack, starri...
Name: mutant, dtype: object

In [89]:
dfm[0] = dfm["label"]
dfm[1] = dfm["mutant"]
dfm["template"] = dfm["original"]
dfm = dfm.drop(columns=["original", "mutant", "gender", "label"])
# dfm = dfm.drop(columns=["mutant", "gender", "label"])

dff[0] = dff["label"]
dff[1] = dff["mutant"]
dff["template"] = dff["original"]
dff = dff.drop(columns=["original", "mutant", "gender", "label"])
# dff = dff.drop(columns=["mutant", "gender", "label"])

In [90]:
dfm

,0,1,template
0,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
1,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
2,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
3,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
4,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
...,...,...,...
138995,1,"First, I'm a huge Justin fan. I grew up knowin...","First, I'm a huge Buddy Holly fan. I grew up k..."
138996,1,"First, I'm a huge Terrence fan. I grew up know...","First, I'm a huge Buddy Holly fan. I grew up k..."
138997,1,"First, I'm a huge Roger fan. I grew up knowing...","First, I'm a huge Buddy Holly fan. I grew up k..."
138998,1,"First, I'm a huge Torrance fan. I grew up know...","First, I'm a huge Buddy Holly fan. I grew up k..."


In [91]:
dfm.to_csv("../data/imdb_mutant/male/test.csv", index=None, header=None, sep="\t")
dff.to_csv("../data/imdb_mutant/female/test.csv", index=None, header=None, sep="\t")